In [13]:
"""
lookup candidate entities and classes
"""
import os
import pandas as pd
import sys
import argparse
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

current_path = os.getcwd()
parser = argparse.ArgumentParser()
parser.add_argument(
    '--input_dir',
    type=str,
    default=os.path.join(current_path, 'data'),
    help='Directory of input/output')
parser.add_argument(
    '--file_type',
    type=str,
    default='csv',
    help='File type')
parser.add_argument(
    '--lookup_results_rank',
    type=int,
    default=5,
    help='File type')

FLAGS, unparsed = parser.parse_known_args()
# if not os.path.exists(FLAGS.input_dir):
#     os.mkdir(FLAGS.input_dir)


In [14]:
# Get all the csv files from the input directory
def get_data_files(data_folder):
    """
    A function used to get all the csv files from the input directory
    ...

    Attributes
    ----------
    data_folder : str
        the folder within  the working directory where the data is located
    """

    files = [] # a list of all filenames, including file extensions, that contain data
    csv_files = [] # same list as above but without the file extension

    # Get the list of files
    files = [f for f in os.listdir(FLAGS.input_dir+data_folder) if os.path.isfile(os.path.join(FLAGS.input_dir+data_folder, f))]
    csv_files = [f.replace(".csv","") for f in os.listdir(FLAGS.input_dir+data_folder) if os.path.isfile(os.path.join(FLAGS.input_dir+data_folder, f))]
    
    return csv_files

def get_target_cta_columns(target_config_file, data_folder, csv_files, filter_col = True):
    """
    A function used to get which columns from the csv files need to be considered for the CTA. This is a subset of the file columns ignoring anything that is not an entity
    ...

    Attributes
    ----------
    target_config_file : str
        the file that contains the target column indices for each file
    csv_files : list
        the list of csv files that have the tabular data
    filter_col : boolean
        a flag to indicate whether we should narrow down the reading of the columns to only those targeted for the CTA task
    """
   
    target_col_file = os.path.join(FLAGS.input_dir+data_folder, target_config_file)
    df_target_col = pd.read_csv(target_col_file,header=None, names=['filename','column_index'])
    
    # filter to only those files that are included in the csv_files
    df_target_col = df_target_col.loc[df_target_col['filename'].isin(csv_files)]
    
    # collapse all rows pertaining to the same file into one key value pair. The key is the filename and the value is the list with the column indices that should be considered
    # dict_target = {'CTRL_DBP_GEO_european_countries_capital_populated_cities': [0, 1, 2]}
    dict_target = dict()
    
    for index,row in df_target_col.iterrows():
        
        # is this is the first row with this file create the key
        if row['filename'] not in dict_target:
            dict_target[row['filename']]= []
            
        # append the new target column to the target column list for that file
        if filter_col:
            dict_target[row['filename']].append(int(row['column_index']))
    
    return dict_target

def get_ground_truth(file, folder, csv_files):
    """
    A function used to get the ground truths as provided in the setup
    ...

    Attributes
    ----------
    file : str
        the file that contains the ground truth for the class of each column in each file
    folder : str
        the folder that contains the ground truth file
    csv_files : list
        the list of csv files that have the tabular data
    """
    
    dbo_prefix = 'http://dbpedia.org/ontology/'
   
    filepath = os.path.join(FLAGS.input_dir+folder, file)
    df_ground_truth = pd.read_csv(filepath,header=None, names=['filename','column_index', 'class'])
    
    # filter to only those files that are included in the csv_files
    df_ground_truth = df_ground_truth.loc[df_ground_truth['filename'].isin(csv_files)]
    
    # collapse all rows pertaining to the same file into one key value pair. The key is the filename and the value is the list with the column indices that should be considered
    # dict_target = {'CTRL_DBP_GEO_european_countries_capital_populated_cities': [0, 1, 2]}
    dict_gt = dict()
    
    for index,row in df_ground_truth.iterrows():
        
        # is this is the first row with this file create the key
        if row['filename'] not in dict_gt:
            dict_gt[row['filename']]= dict()
            
        # append the new target column to the target column list for that file
        dict_gt[row['filename']][int(row['column_index'])] = row['class'].split(dbo_prefix)[1]
    
    return dict_gt

def read_data(data_folder, dict_target_col, has_header_row = False):
    """
    A function used to read the data from the csvs in the data_folder only considering the columns that are in the dict_target_col
    ...

    Attributes
    ----------
    folder : str
        the folder that contains the csvs with the tabular data
    dict_target_col : dictionary
        a dictionary with csv filenames as the key and an array of relevant column indices as a value
    has_header_row : boolean
        a flag to indicate whether the first row in the csv files needs to be skipped as it is a header
    """
    data = list()

    for file in dict_target_col:
        element = dict()
        element['filename'] = file
        df_data = pd.DataFrame()
        df_title = pd.DataFrame()



        filename = file + '.' + FLAGS.file_type
        tab_data_file = os.path.join(FLAGS.input_dir + data_folder, filename)

        # read the file data in a dataframe. Also read the column titles if we need to use them
        if len(dict_target_col[file])>0:
            if has_header_row:
                df_data = pd.read_csv(tab_data_file,header=None, skiprows=[0], usecols=dict_target_col[file])
                df_title = pd.read_csv(tab_data_file,header=None, usecols=dict_target_col[file], nrows = 1)
            else:
                df_data = pd.read_csv(tab_data_file,header=None, usecols=dict_target_col[file])
        else:
            if has_header_row:
                df_data = pd.read_csv(tab_data_file,header=None, skiprows=[0])
                df_title = pd.read_csv(tab_data_file,header=None, nrows = 1)
            else:
                df_data = pd.read_csv(tab_data_file,header=None)

        # add the column headers to the data dictionary
        try:
            element['column_titles'] = list(df_title.iloc[0,:])
        except:
            pass

        file_element = dict()
        for column in df_data.columns:
            file_element[column] = list(set(df_data[column]))
        element['data'] = file_element

        element['dataframe'] = df_data    
        data.append(element)
    
    return data

### Background Setup

As part of this initial step we will need to load the data we are going to process as well as the targets we are trying to meet. The data is located in the data folder as follows
- round_1:
    - gt: the expected outcome (ground truth)
    - tables: the tabular data
    - targets: the columns / cells we need to consider for the CTA/CEA
----
Step 1: Get a list of all the csv files in the data folder

In [50]:
# Get the list of csv files with tabular data
csv_files = get_data_files('\\round_1\\tables')
# csv_files = csv_files[:1]
csv_files

['10579449_0_1681126353774891032',
 '11833461_1_3811022039809817402',
 '13719111_1_5719401842463579519',
 '14067031_0_559833072073397908',
 '1438042986423_95_20150728002306-00125-ip-10-236-191-2_88435628_5',
 '1438042986423_95_20150728002306-00329-ip-10-236-191-2_805336391_10',
 '1438042989018_40_20150728002309-00067-ip-10-236-191-2_57714692_2',
 '1438042989043_35_20150728002309-00287-ip-10-236-191-2_875026214_2',
 '14380604_4_3329235705746762392',
 '16767252_0_2409448375013995751',
 '20135078_0_7570343137119682530',
 '21245481_0_8730460088443117515',
 '21362676_0_6854186738074119688',
 '22864497_0_8632623712684511496',
 '24036779_0_5608105867560183058',
 '25404227_0_2240631045609013057',
 '26310680_0_5150772059999313798',
 '28086084_0_3127660530989916727',
 '29414811_12_251152470253168163',
 '29414811_13_8724394428539174350',
 '29414811_2_4773219892816395776',
 '29414811_6_8221428333921653560',
 '33401079_0_9127583903019856402',
 '34041816_1_4749054164534706977',
 '35188621_0_60585531

Step 2: Get the columns we need to consider for the CTA task

In [51]:
# Get the columns we need to consider for the CTA task
dict_target_col = get_target_cta_columns('CTA_Round1_Targets.csv', '\\round_1\\targets', csv_files,True)
dict_target_col

{'58891288_0_1117541047012405958': [1, 3],
 '8468806_0_4382447409703007384': [1, 2],
 '50245608_0_871275842592178099': [0, 3, 4],
 '14067031_0_559833072073397908': [1, 7, 5, 0],
 '39759273_0_1427898308030295194': [1, 3],
 '14380604_4_3329235705746762392': [1, 2],
 '20135078_0_7570343137119682530': [3, 1],
 '29414811_6_8221428333921653560': [1, 4, 2],
 '34041816_1_4749054164534706977': [2, 1],
 '29414811_2_4773219892816395776': [1, 4, 2],
 '99070098_0_2074872741302696997': [1],
 '35188621_0_6058553107571275232': [3, 1],
 '43237185_1_3636357855502246981': [3, 0, 2],
 '46671561_0_6122315295162029872': [0, 1],
 '11833461_1_3811022039809817402': [0, 1],
 '21245481_0_8730460088443117515': [1, 0],
 '38428277_0_1311643810102462607': [3, 1],
 '13719111_1_5719401842463579519': [0],
 '77694908_0_6083291340991074532': [1, 3],
 '88523363_0_8180214313099580515': [0, 2],
 '1438042989043_35_20150728002309-00287-ip-10-236-191-2_875026214_2': [0],
 '25404227_0_2240631045609013057': [3, 1],
 '9475172_1_1

Step 3: Get the ground truth for all columns in the set of csv files

In [52]:
ground_truth = get_ground_truth('CTA_Round1_gt.csv', '\\round_1\\gt', csv_files)
ground_truth

{'58891288_0_1117541047012405958': {1: 'Film', 3: 'Person'},
 '8468806_0_4382447409703007384': {1: 'Lake', 2: 'Country'},
 '50245608_0_871275842592178099': {0: 'Film', 3: 'Person', 4: 'Writer'},
 '14067031_0_559833072073397908': {1: 'Language',
  7: 'Currency',
  5: 'City',
  0: 'Country'},
 '39759273_0_1427898308030295194': {1: 'Film', 3: 'Person'},
 '14380604_4_3329235705746762392': {1: 'Company', 2: 'PopulatedPlace'},
 '20135078_0_7570343137119682530': {3: 'Person', 1: 'Film'},
 '29414811_6_8221428333921653560': {1: 'VideoGame', 4: 'Company', 2: 'Genre'},
 '34041816_1_4749054164534706977': {2: 'City', 1: 'Airport'},
 '29414811_2_4773219892816395776': {1: 'VideoGame', 4: 'Company', 2: 'Genre'},
 '99070098_0_2074872741302696997': {1: 'Mountain'},
 '35188621_0_6058553107571275232': {3: 'Person', 1: 'Film'},
 '43237185_1_3636357855502246981': {3: 'Scientist',
  0: 'Scientist',
  2: 'EducationalInstitution'},
 '46671561_0_6122315295162029872': {0: 'VideoGame', 1: 'Genre'},
 '11833461_1_3

# Load Data

The next step is to load the data from the csv files. We load the data as an array of dictionaries.
Each dictionary will have the following structure:<br>
{<br>
<blockquote>
<strong>'filename':</strong> '1438042986423_95_20150728002306-00125-ip-10-236-191-2_88435628_5',<br>
<strong>'column_titles'</strong>: ['Party'],<br>
<strong>'data'</strong>: <br>
    {<br>
    <blockquote>
        <strong>0:</strong> ['PC', 'Lib-Dem','SNP','UKIP','Labour','BNP','Conservative','Green']<br>
    </blockquote>
        },<br>

<strong>'dataframe':</strong>               0
 0  Conservative
 1        Labour
 2       Lib-Dem
 3           SNP
 4            PC
 5         Green
 6           BNP
 7          UKIP<br>
</blockquote>    
 }


In [53]:
data = read_data('\\round_1\\tables', dict_target_col,True)

In [54]:
import requests
import xml.etree.ElementTree as ET
import time

def retrieve_dbpedia_classes (query_string, max_hits = 5):
    web_api = 'http://lookup.dbpedia.org/api/search/KeywordSearch?MaxHits=%s&QueryString=%s'
    dbo_prefix = 'http://dbpedia.org/ontology/'
    dbp_prefix = 'http://dbpedia.org/resource/'
    entity_classes = dict()
    try:
        lookup_url = web_api % (max_hits, query_string)
#         print(lookup_url)
        lookup_res = requests.get(lookup_url)
        root = ET.fromstring(lookup_res.content)
        i=0
        for child in root:
            i+=1
#             print("\n")
            entity = child[1].text.split(dbp_prefix)[1]
#             print(entity)
            classes = list()
            for cc in child[3]:
                cls_URI = cc[1].text
#                 print(cls_URI)
                if dbo_prefix in cls_URI:
                    classes.append((cls_URI.split(dbo_prefix)[1]))
            if len(classes)>0:
                entity_classes[entity] = dict()
                entity_classes[entity]['rank'] = i
                entity_classes[entity]['candidate_classes'] = classes
    except UnicodeDecodeError:
        pass
    return entity_classes

## Lookup cell_values

With the data loaded in the *data* dictionary the next step is to lookup the cell values in the DBpedia endpoint and get the canidate classes and entities.
Each cell value is only looked up once, however we still keep track of any column it might have appeared in as well as all candidate entities and classes it may have matched to.

For this level of analysis we are flexible to store the 5 top lookup results for each cell value (default value for FLAGS.lookup_results_rank).
We will then assess the number of classifiers we need to train later and perhaps filter out any candidate classes that only appeared in lower ranks.

The outcome of the lookup is stored in the *cell_values* dictionary as follows:

{<strong>"Madagascar":</strong><br>
{
<blockquote><strong>"location":</strong> [("14067031_0_559833072073397908",0)]
            , <br><strong>"candidate_entities":</strong><br> 
                        {
                            <blockquote><strong>"Madagascar":</strong> <br>{<blockquote><strong>"rank":</strong> 1,<br> <strong>"candidate_classes":</strong> ["Place", "Country", "PopulatedPlace", "Location"]</blockquote>}, <br>
                            <strong>"Antananarivo":</strong> <br> {<blockquote><strong>"rank":</strong> 3,<br> <strong>"candidate_classes":</strong> ["Settlement", "Place", "PopulatedPlace", "Location"]</blockquote>}, <br>
                            <strong>"List_of_Madagascar_(franchise)_characters":</strong> <br> {<blockquote><strong>"rank":</strong> 4,<br> <strong>"candidate_classes":</strong> ["FictionalCharacter", "Agent"]</blockquote>},<br>
                            <strong>"Madagascar_national_football_team"</strong> <br> {<blockquote><strong>"rank":</strong> 5,<br> <strong>"candidate_classes":</strong> ["Organisation", "SoccerClub", "Agent", "SportsClub"]</blockquote>}<br>
</blockquote>}<br> 
</blockquote>},<br>
               
 <strong>"South Africa":</strong> {...},<br>
  ...<br>
 }

In [56]:
cell_values = dict()
i = 0

from IPython.display import clear_output

size = 0
for file_i in range(len(data)):
    for col in data[file_i]['data']:
        for line_j in range(len(data[file_i]['data'][col])):
            size+=1
            
start_time = time.time()

# from tqdm import tqdm
for file_i in range(len(data)):
#     print(data[file_i])
    filename = data[file_i]['filename']
    for col in data[file_i]['data']:
        column_index = col
#         print(col)
#         print(data[file_i]['data'][col])
        for line_j in range(len(data[file_i]['data'][col])):
            i+=1
            cell_value = data[file_i]['data'][col][line_j]
            clear_output(wait=True)
            print('{0:.2f}'.format(100*i/size,2),'-->',filename, ": ",cell_value)
            if cell_value in cell_values.keys():
                cell_values[cell_value]['location'].append((filename,column_index))
            else:
                cell_values[cell_value] = dict()
                cell_values[cell_value]['location'] = [(filename,column_index)]
                try:
                    cell_values[cell_value]['candidate_entities'] = retrieve_dbpedia_classes(cell_value.replace("[",'').replace("]",''),FLAGS.lookup_results_rank)
                except:
                    cell_values[cell_value]['candidate_entities'] = retrieve_dbpedia_classes(cell_value,FLAGS.lookup_results_rank)
                
end_time = time.time()

print(f"{int(end_time - start_time)//60} min and {int((end_time - start_time)%60)} seconds Elapsed")

100.00 --> 69537082_0_7789694313271016902 :  Northern Mariana Islands
88 min and 41 seconds Elapsed


In [81]:
# cell_values

In [57]:
import json

# Store the dictionary with the lookup results for each cell value in the tabular data
with open('cell_values.json', 'w') as fp:
    json.dump(cell_values, fp)

In [141]:
retrieve_dbpedia_classes('Steven Spielberg',FLAGS.lookup_results_rank)

{'Taken_(miniseries)': {'rank': 5,
  'candidate_classes': ['TelevisionShow', 'Work']}}

In [46]:
# Load the dictionary with the lookup results for each cell value in the tabular data
with open('cell_values.json') as json_file:
    loaded_cell_values = json.load(json_file)

In [92]:
# dict_target_col

filename = '58891288_0_1117541047012405958'
temp = {'58891288_0_1117541047012405958': [1, 3]}
temp_cell_values = dict()
temp_cell_values['The Bridge on the River Kwai'] = cell_values['The Bridge on the River Kwai'] 

temp_cell_values

{'The Bridge on the River Kwai': {'location': [('58891288_0_1117541047012405958',
    1),
   ('77694908_0_6083291340991074532', 1),
   ('25404227_0_2240631045609013057', 1),
   ('16767252_0_2409448375013995751', 1)],
  'candidate_entities': {'The_Bridge_over_the_River_Kwai': {'rank': 1,
    'candidate_classes': ['Book', 'WrittenWork', 'Work']},
   'Kanchanaburi_Province': {'rank': 3,
    'candidate_classes': ['AdministrativeRegion',
     'Place',
     'PopulatedPlace',
     'Location',
     'Region']},
   'Balu_Mahendra': {'rank': 4, 'candidate_classes': ['Person', 'Agent']},
   'A._B._Raj': {'rank': 5, 'candidate_classes': ['Person', 'Agent']}}}}

In [136]:
dict_col_candidate_classes = dict()
threshold = 3

for filename in dict_target_col: #later replace with dict_target_col
    dict_col_candidate_classes[filename] = dict()
    for i in dict_target_col[filename]:
        dict_col_candidate_classes[filename][i] = []
#     print(key)
    for cell_value in cell_values:
        try:
            column_index = dict(cell_values[cell_value]['location'])[filename]
            for candidate_entity in cell_values[cell_value]['candidate_entities']:
#                 print(candidate_entity)
                rank = cell_values[cell_value]['candidate_entities'][candidate_entity]['rank']
                if rank <= threshold:
                    for candidate_class in cell_values[cell_value]['candidate_entities'][candidate_entity]['candidate_classes']:
                        dict_col_candidate_classes[filename][column_index].append((candidate_class, candidate_entity, cell_value,rank))
                
            dict_col_candidate_classes[filename][column_index]
#             print('found')
        except:
            pass
#             print('not found')

In [142]:
cell_values['Steven Spielberg']

{'location': [('58891288_0_1117541047012405958', 3),
  ('39759273_0_1427898308030295194', 3),
  ('20135078_0_7570343137119682530', 3),
  ('35188621_0_6058553107571275232', 3),
  ('38428277_0_1311643810102462607', 3),
  ('77694908_0_6083291340991074532', 3),
  ('25404227_0_2240631045609013057', 3),
  ('84548468_0_5955155464119382182', 3),
  ('69881946_0_1105130426898457358', 3),
  ('53822652_0_5767892317858575530', 3),
  ('21362676_0_6854186738074119688', 3)],
 'candidate_entities': {'Taken_(miniseries)': {'rank': 5,
   'candidate_classes': ['TelevisionShow', 'Work']}}}

In [137]:
dict_target_col['20135078_0_7570343137119682530']

[3, 1]

In [138]:
dict_col_candidate_classes['20135078_0_7570343137119682530'][3]

[('Person', 'Jean_Renoir', 'Jean Renoir', 1),
 ('Agent', 'Jean_Renoir', 'Jean Renoir', 1),
 ('Film', 'The_Little_Theatre_of_Jean_Renoir', 'Jean Renoir', 3),
 ('Work', 'The_Little_Theatre_of_Jean_Renoir', 'Jean Renoir', 3),
 ('Person', 'Nicholas_Ray', 'Nicholas Ray', 1),
 ('Agent', 'Nicholas_Ray', 'Nicholas Ray', 1),
 ('Politician', 'Nicholas_Winterton', 'Nicholas Ray', 2),
 ('Person', 'Nicholas_Winterton', 'Nicholas Ray', 2),
 ('Agent', 'Nicholas_Winterton', 'Nicholas Ray', 2),
 ('MusicalArtist', 'Nicholas_Furlong_(musician)', 'Nicholas Ray', 3),
 ('Person', 'Nicholas_Furlong_(musician)', 'Nicholas Ray', 3),
 ('Artist', 'Nicholas_Furlong_(musician)', 'Nicholas Ray', 3),
 ('Agent', 'Nicholas_Furlong_(musician)', 'Nicholas Ray', 3),
 ('Person', 'Alfred_Hitchcock', 'Alfred Hitchcock', 1),
 ('Agent', 'Alfred_Hitchcock', 'Alfred Hitchcock', 1),
 ('TelevisionShow', 'Alfred_Hitchcock_Presents', 'Alfred Hitchcock', 2),
 ('Work', 'Alfred_Hitchcock_Presents', 'Alfred Hitchcock', 2),
 ('Person', 

In [29]:
candidate_classes = list([])
candidate_classes_rank = list([])
for key in entity_classes:
    for candicate_class in entity_classes[key]:
        cc,rank = candicate_class
        if cc not in candidate_classes:
            candidate_classes.append(cc)
            candidate_classes_rank.append((cc,rank))

candidate_classes_rank = sorted(candidate_classes_rank, key=lambda x: x[1])
[t[0] for t in candidate_classes_rank if t[1]<=3]

['AdministrativeRegion',
 'Place',
 'PopulatedPlace',
 'Location',
 'Region',
 'Settlement',
 'Town',
 'HistoricPlace',
 'Criminal',
 'Person',
 'SportsTeam',
 'AmericanFootballTeam',
 'Building',
 'ArchitecturalStructure',
 'Village',
 'River',
 'Stream',
 'Lake',
 'Organisation',
 'School',
 'EducationalInstitution',
 'Agent',
 'SoccerLeague',
 'SportsLeague',
 'SoccerClub',
 'SportsClub',
 'Politician']

In [2]:
spark = SparkSession.builder.appName('Annotation').getOrCreate()

In [5]:
spark

In [13]:
df_pyspark = spark.read.option('header','True').csv('CTRL_DBP_GEO_us_lakes.csv')

In [18]:
df_pyspark.printSchema()

root
 |-- lake: string (nullable = true)
 |-- area: string (nullable = true)
 |-- location: string (nullable = true)
 |-- state: string (nullable = true)



In [22]:
type(df_pyspark)

NameError: name 'df_pyspark' is not defined